In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("data/data_intents.csv")
df.head()

,intent_name,examples,label
0,despedida,adios,0
1,despedida,buenas noches,0
2,despedida,nos vemos luego,0
3,despedida,que tengas buen dia,0
4,despedida,chau,0


In [2]:
print(df['intent_name'].value_counts()[10:30])

matricula__cronograma                                43
matricula_rezagada__procedimiento                    43
carnet_universitario__solicitud                      42
constancia_de_notas__procedimiento                   42
retiro_total__fechas                                 42
reserva_matricula__procedimiento                     42
retiro_parcial__maximo_cursos                        42
retiro_parcial__unico_curso                          41
reincorporacion__procedimiento                       41
retiro_total__procedimiento                          41
reserva_matricula__fechas                            41
retiro_total__requisitos_documentos_justificacion    41
reincorporacion__fechas                              41
pago_autoseguro_procedimiento                        41
matricula__horarios                                  41
cursos_horarios                                      40
constancia_matricula__solicitud_procedimiento        40
constancia_de_estudios__procedimiento           

In [3]:
def convert_dframe_list(df):
    data_dict = []
    for index ,row in df.iterrows():
        sample = { 
                "text": row['examples'],
                "intent_name": row['intent_name'],
                "entities": []
                }
        data_dict.append(sample)
    return data_dict

data_dict =  convert_dframe_list(df)
data_dict

[{'text': 'adios', 'intent_name': 'despedida', 'entities': []},
 {'text': 'buenas noches', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'que tengas buen dia', 'intent_name': 'despedida', 'entities': []},
 {'text': 'chau', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos', 'intent_name': 'despedida', 'entities': []},
 {'text': 'te veo luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego Aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego que tengas buen dia',
  'intent_name': 'despedida',
  'entities': []},
 {'text': 'adios que tenga lindo dia',
  'intent_name': 'despedida',
  'entities': []},
 {'text': 'nos vemos aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'Adios aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos pronto', 'inte

In [4]:
pd.DataFrame(data_dict)

,text,intent_name,entities
0,adios,despedida,[]
1,buenas noches,despedida,[]
2,nos vemos luego,despedida,[]
3,que tengas buen dia,despedida,[]
4,chau,despedida,[]
...,...,...,...
1779,¿Cuanto porciento al menos deben coincidir el ...,fuera_alcance,[]
1780,¿Una asignatura electiva se cerrará si el núme...,fuera_alcance,[]
1781,¿Cuanto es la cantidad mínima de estudiantes p...,fuera_alcance,[]
1782,¿Quienes podrán matricularse en las asignatura...,fuera_alcance,[]


In [5]:
entities_table = { "procedimiento":[
            "retiro parcial",
            "retiro total",
            "reserva de matricula",
            "reincorporación",
            "reincorporacion"],
            "constancia":["constancia de matricula",
            "Constancia de Matrícula",
            "constancia de notas",
            "constancia de estudios",
            "constacia de estudios"]
    }
entities_table

{'procedimiento': ['retiro parcial',
  'retiro total',
  'reserva de matricula',
  'reincorporación',
  'reincorporacion'],
 'constancia': ['constancia de matricula',
  'Constancia de Matrícula',
  'constancia de notas',
  'constancia de estudios',
  'constacia de estudios']}

In [6]:
import re

def label_entities_text(text, entities_table):
    text_lower = text.lower()
    text_label_entities = text
    for label, ent_values in entities_table.items():
        for ent in ent_values:
            match = re.search(ent,text_lower)
            if match:
                start_span, end_span = match.span()
                val_ent = text[start_span:end_span]
                text_label_entities = text.replace(val_ent, f"<{val_ent}>[{label}]") #
    return text_label_entities

def extract_entities(text):

    entities = []
    match = re.search(r"<[^>]+>\[[^\]]+\]",text)
    while match:
        ent_block = match.group(0)
        span_block = match.span()
        
        ## Extract Entity value
        match_ent_val =  re.search("<.+>",ent_block)
        ent_val = match_ent_val.group(0)[1:-1]
        # Extract Entity Name
        match_ent_name =  re.search("\[.+\]",ent_block)
        ent_name = match_ent_name.group(0)[1:-1]
        
        text = text.replace(ent_block, f"{ent_val}",1)
        entities.append({
            "value": ent_val, 
            "entity": ent_name, 
            "span_start": span_block[0] , 
            "span_end": span_block[0] + len(ent_val)})
    
        match = re.search(r"<[^>]+>\[[^\]]+\]",text)

    return text, entities

In [7]:
new_data_dict = []

for item in data_dict:
    text = item['text']
    intent_name = item['intent_name']
    new_text = label_entities_text(text, entities_table)
    sample = { 
        "text": new_text,
        "intent_name": item['intent_name']
        }
    new_data_dict.append(sample)

data_nlu = pd.DataFrame(new_data_dict)

data_nlu.to_csv('data/data_nlu.csv', index= False)

In [8]:
pd.DataFrame(data_dict)

,text,intent_name,entities
0,adios,despedida,[]
1,buenas noches,despedida,[]
2,nos vemos luego,despedida,[]
3,que tengas buen dia,despedida,[]
4,chau,despedida,[]
...,...,...,...
1779,¿Cuanto porciento al menos deben coincidir el ...,fuera_alcance,[]
1780,¿Una asignatura electiva se cerrará si el núme...,fuera_alcance,[]
1781,¿Cuanto es la cantidad mínima de estudiantes p...,fuera_alcance,[]
1782,¿Quienes podrán matricularse en las asignatura...,fuera_alcance,[]


In [9]:
data_dict

[{'text': 'adios', 'intent_name': 'despedida', 'entities': []},
 {'text': 'buenas noches', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'que tengas buen dia', 'intent_name': 'despedida', 'entities': []},
 {'text': 'chau', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos', 'intent_name': 'despedida', 'entities': []},
 {'text': 'te veo luego', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego Aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'hasta luego que tengas buen dia',
  'intent_name': 'despedida',
  'entities': []},
 {'text': 'adios que tenga lindo dia',
  'intent_name': 'despedida',
  'entities': []},
 {'text': 'nos vemos aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'Adios aerito', 'intent_name': 'despedida', 'entities': []},
 {'text': 'nos vemos pronto', 'inte

In [10]:

for item in data_dict:
    text = item['text']
    text_label_entities = label_entities_text(text, entities_table)
    #item['text_label_entities'] = text_label_entities
    text_formatted, entities =  extract_entities(text_label_entities)
    item['text'] = text_formatted
    item['entities'] = entities

In [12]:
def extract_entities2(text):

    entities = []
    match = re.search(r"<[^>]+>\[[^\]]+\]",text)
    while match:
        ent_block = match.group(0)
        span_block = match.span()
        
        ## Extract Entity value
        match_ent_val =  re.search("<.+>",ent_block)
        ent_val = match_ent_val.group(0)[1:-1]
        # Extract Entity Name
        match_ent_name =  re.search("\[.+\]",ent_block)
        ent_name = match_ent_name.group(0)[1:-1]
        
        text = text.replace(ent_block, f"{ent_val}",1)
        entities.append({
            "value": ent_val, 
            "entity": ent_name, 
            "span_start": span_block[0] , 
            "span_end": span_block[0] + len(ent_val)})
    
        match = re.search(r"<[^>]+>\[[^\]]+\]",text)

    return text, entities

In [13]:
text = "hay algun formato especial para el <retiro total>[procedimiento]?"
extract_entities2(text)

('hay algun formato especial para el retiro total?',
 [{'value': 'retiro total',
   'entity': 'procedimiento',
   'span_start': 35,
   'span_end': 47}])

In [14]:
text = "hay algun formato especial para el retiro total?"


In [15]:
import json
def save_json(data_dict, name = "data_nlu"):
    with open(f"{name}.json", "w", encoding='utf-8') as f:
        json.dump(data_dict, f, indent= 4)

def load_json(path = "./data_nlu.json"):
    with open(f"{path}", "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

save_json(data_dict, name = "data/data_nlu")

In [29]:
from sklearn.model_selection import train_test_split
intents_labels = [sample['intent_name'] for sample in data_dict]

train_data , test_data = train_test_split(data_dict, random_state=42, test_size= 0.2, shuffle= True, stratify= intents_labels)
print("train size:", len(train_data))

print("test size:", len(test_data))

save_json(train_data, name = "data/train_nlu")

save_json(test_data, name = "data/test_nlu")

train size: 1427
test size: 357


In [ ]:
text = "no tengo ningun documento que validar mi retiro total que puedo hacer?"